### Import Python libraries/packages

In [1]:
import numpy as np                                      # Scientific computing package
import numpy.ma as ma                                   # Import numpy mask
import pandas as pd                                     # Working with dataframes
import requests                                         # API calls
import os                                               # File handling
import re                                               # Python RegEx
import rasterio as rio                                  # Working with geospatial raster data(GeoTiff)
from rasterio.mask import mask                          # To mask raster files with shapes
from shapely.geometry import Polygon                    # Import Shapely to work with polygons
import plotly.graph_objects as go                       # Import Plotly library for 3D plotting

### Try different addresses in city of Ghent to make a selection of Canopy Height Model Geotiff files 

In [11]:
# Retrieve Lambert 72 coordinates from address with API call
def retrieve_coords(address):
    req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1")
    x = req.json()['LocationResult'][0]['Location']['X_Lambert72']
    y = req.json()['LocationResult'][0]['Location']['Y_Lambert72']
    print(f"Lambert72 coords of provided address: \n x= {x} y= {y}")
    return x,y

# Retrieve CHM map that contains these coordinates from CSV with bounding box coordinates per CHM map
def retrieve_map(x,y):
    maps = pd.read_csv('/home/becode/AI/3dhouse/chm.csv', sep=',')
    maps = maps.filename[(maps.left < x) & (x < maps.right) & (maps.bottom < y) & (y < maps.top)]
    path_open = maps.iloc[0]
    print(f"map retrieved from : {path_open}")
    return path_open

In [12]:
addresses = ['Tortelduifstraat 22, 9000 Gent','Ooievaarstraat 94, 9000 Gent','Houtemlaan 21, 9000 Gent','Stormvogelstraat 14, 9000 Gent',
           'Ebergiste De Deynestraat 2B, 9000 Gent', 'Louis Roelandtplein 15, 9000 Gent','Corneel Heymanslaan 20, 9000 Gent',
           'Ottergemsesteenweg 183, 9000 Gent', 'Gangmakersstraat 18, 9000 Gent', 'Ter Platen 5, 9000 Gent','Mageleinstraat 82, 9000 Gent',
            'Nieuwhof 68, 9000 Gent']

In [13]:
for address in addresses:
    print(f"-> address: {address}")
    x, y = retrieve_coords(address)
    retrieve_map(x,y)

-> address: Tortelduifstraat 22, 9000 Gent
Lambert72 coords of provided address: 
 x= 102479.59 y= 195425.6
map retrieved from : tile_chm_k22_68.tif
-> address: Ooievaarstraat 94, 9000 Gent
Lambert72 coords of provided address: 
 x= 102946.11 y= 194450.68
map retrieved from : tile_chm_k22_100.tif
-> address: Houtemlaan 21, 9000 Gent
Lambert72 coords of provided address: 
 x= 102776.78 y= 193936.4
map retrieved from : tile_chm_k22_132.tif
-> address: Stormvogelstraat 14, 9000 Gent
Lambert72 coords of provided address: 
 x= 102569.99 y= 192071.03
map retrieved from : tile_chm_k22_164.tif
-> address: Ebergiste De Deynestraat 2B, 9000 Gent
Lambert72 coords of provided address: 
 x= 104072.6 y= 189868.08
map retrieved from : tile_chm_k22_262.tif
-> address: Louis Roelandtplein 15, 9000 Gent
Lambert72 coords of provided address: 
 x= 104694.48 y= 190155.4
map retrieved from : tile_chm_k22_230.tif
-> address: Corneel Heymanslaan 20, 9000 Gent
Lambert72 coords of provided address: 
 x= 105132.

#### The range of maps is too big. Every CHM raster file is 4 Mb in size so I will look for maps only in city centre of Ghent within postal code 9000 rang

### Narrow down to look for address in the city centre in postal code 9000 Ghent

In [14]:
addresses = ['Prudens Van Duyseplein 1, 9000 Gent', 'Tentoonstellingslaan 65, 9000 Gent', 'Kouter 5, 9000 Gent',
            'Voldersstraat 1, 9000 Gent','Penitentenstraat 28, 9000 Gent', 'Kartuizerlaan 135, 9000 Gent',
            'Sint-Lievenspoortstraat 248, 9000 Gent', 'Lange Violettestraat 259, 9000 Gent', 
            'Snoekstraat 35, 9000 Gent','Ossenstraat 63, 9000 Gent','Franklin Rooseveltlaan 102, 9000 Gent',
             'Sint-Annaplein 16, 9000 Gent', 'Keizer Karelstraat 1, 9000 Gent',
            'Lucas Munichstraat 38, 9000 Gent','Gebroeders Vandeveldestraat 101, 9000 Gent', 'Sint-Hubertusstraat 21, 9000 Gent',
            'Oude Houtlei 78, 9000 Gent', 'Burgstraat 118, 9000 Gent']

In [15]:
for address in addresses:
    print(f"-> address: {address}")
    x, y = retrieve_coords(address)
    retrieve_map(x,y)

-> address: Prudens Van Duyseplein 1, 9000 Gent
Lambert72 coords of provided address: 
 x= 104612.22 y= 192696.02
map retrieved from : tile_chm_k22_166.tif
-> address: Tentoonstellingslaan 65, 9000 Gent
Lambert72 coords of provided address: 
 x= 105437.71 y= 192580.96
map retrieved from : tile_chm_k22_167.tif
-> address: Kouter 5, 9000 Gent
Lambert72 coords of provided address: 
 x= 104874.47 y= 193537.8
map retrieved from : tile_chm_k22_134.tif
-> address: Voldersstraat 1, 9000 Gent
Lambert72 coords of provided address: 
 x= 104861.67 y= 193743.43
map retrieved from : tile_chm_k22_134.tif
-> address: Penitentenstraat 28, 9000 Gent
Lambert72 coords of provided address: 
 x= 104983.72 y= 194440.28
map retrieved from : tile_chm_k22_102.tif
-> address: Kartuizerlaan 135, 9000 Gent
Lambert72 coords of provided address: 
 x= 105042.13 y= 194973.05
map retrieved from : tile_chm_k22_103.tif
-> address: Sint-Lievenspoortstraat 248, 9000 Gent
Lambert72 coords of provided address: 
 x= 105664.77

#### I will select geotiff files:   adjacent k22_102, k22_103, k22_104, adjacent k22_134, k22_135, k22_136, adjacent k22_166, k22_167, k22_168
#### They will create a 3X3 grid for the city centre of Ghent

### Update earlier code to catch exceptions in address format and to make sure an address requested is in my selection of CHM raster files for city centre Ghent

#### Delete old file path for a CHM file in chm.csv

In [82]:
df = pd.read_csv('/home/becode/AI/3dhouse/chm.csv', index_col = 0, sep = ',')
print(df.head(5))
def split(string):
    last_part = string.split('/')[-1]
    return last_part
df.filename = df.filename.astype(str)
df.filename = df.filename.apply(split)
print(df.head(5))
df = df.to_csv('/home/becode/AI/3dhouse/chm.csv')

                                       filename      left    bottom     right  \
0  /media/becode/EXT/CHMsplit/tile_chm_k1_0.tif  145000.0  246000.0  146000.0   
1  /media/becode/EXT/CHMsplit/tile_chm_k1_1.tif  146000.0  246000.0  147000.0   
2  /media/becode/EXT/CHMsplit/tile_chm_k1_2.tif  147000.0  246000.0  148000.0   
3  /media/becode/EXT/CHMsplit/tile_chm_k1_3.tif  148000.0  246000.0  149000.0   
4  /media/becode/EXT/CHMsplit/tile_chm_k1_4.tif  149000.0  246000.0  150000.0   

        top  
0  247000.0  
1  247000.0  
2  247000.0  
3  247000.0  
4  247000.0  
            filename      left    bottom     right       top
0  tile_chm_k1_0.tif  145000.0  246000.0  146000.0  247000.0
1  tile_chm_k1_1.tif  146000.0  246000.0  147000.0  247000.0
2  tile_chm_k1_2.tif  147000.0  246000.0  148000.0  247000.0
3  tile_chm_k1_3.tif  148000.0  246000.0  149000.0  247000.0
4  tile_chm_k1_4.tif  149000.0  246000.0  150000.0  247000.0


#### List of selected CHM tifs

In [22]:
tif = ['tile_chm_k22_102.tif', 'tile_chm_k22_103.tif', 'tile_chm_k22_104.tif', 
        'tile_chm_k22_134.tif', 'tile_chm_k22_135.tif', 'tile_chm_k22_136.tif',
        'tile_chm_k22_166.tif', 'tile_chm_k22_167.tif', 'tile_chm_k22_168.tif']

#### Updated functions to retrieve coordinates, polygon, masking and plotting

In [23]:
"""
Retrieve Lambert 72 coordinates from an address with API call
"""
def retrieve_coords(address):
    try:
        req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1")
        x = req.json()['LocationResult'][0]['Location']['X_Lambert72']
        y = req.json()['LocationResult'][0]['Location']['Y_Lambert72']
        print("-> Lambert72 coords of provided address: \n x= ", x, " y= ", y)
        return x,y
    except:
        message = "Address coordinates could not be found."
        print(message)
        return message

"""
Retrieve correct Canopy Height Model map to mask with polygon of house shape
"""
def retrieve_map(x,y):
    """
    Create Pandas dataframe from csv file that indexes all splitted CHM raster files and their path, then look
    for the file that contains my x,y Lambert72 coords and return the path from the dataframe
    """
    maps = pd.read_csv('/home/becode/AI/3dhouse/chm.csv', sep=',')
    maps = maps.filename[(maps.left < x) & (x < maps.right) & (maps.bottom < y) & (y < maps.top)]
    path_open = maps.iloc[0]
    
    if path_open in tif:
        filepath = '/home/becode/AI/3dhouse/'
        path = os.path.join(filepath, path_open)
        print(path)
        return path
    else:
        message = "Address coordinates not in selection."
        print(message)
        return message
"""
Retrieve house polygon from address with API call
"""
def retrieve_polygon(address):
    try: 
        result = re.search("^([A-z- ]+)([0-9]+[A-z]?),[ ]?([0-9]+)[ ]?([A-z]+)", address)
        street = result.group(1)
        nb = result.group(2)
        pc = result.group(3)
        city = result.group(4)
        """
        Request polygon from API : https://api.basisregisters.dev-vlaanderen.be/v1/percelen/{objectId} 
        and retrieve .json object
        """
        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
        objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]
        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
        objectId = req["gebouw"]["objectId"]
        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()
        polygon = [req["geometriePolygoon"]["polygon"]]
        print(polygon)
        return polygon
        
    except:
        message = "Polygon for address could not be retrieved."
        print(message)
        return message

def mask_polygon(poly, path):
    """
    Open path of CHM to mask polygon shape from. The polygon is a function parameter as well and is used in the
    mask function as a shape to mask by
    """
    path_chm = path
    map_chm = rio.open(path_chm)
    print('print OK')
    """
    Mask or clip the polygon shape from the CHM raster data with rasterio mask function, this returns a 
    masked raster file and a transform
    """
    mask_chm, mask_chm_transform = mask(dataset=map_chm, shapes=poly, crop=True, nodata=0, filled=True, indexes=1)
    print('-> mask succesful')
    return mask_chm

def dim_house(poly , mask_chm) :
    """
    Create polygon shape with Shapely function Polygon from coordinates of polygon list
    """
    polygon = Polygon(poly[0]['coordinates'][0])
    """
    I need to exclude 0 values from my masked CHM array to calculate a mean height  so I use numpy mask for masking
     the 0's and hereby create the array 'noZeros', sonp.mean(noZeros) will calculate mean height
    """
    noZeros = ma.masked_values(mask_chm, 0)
    """
    Create dict with house dimensions: area of ground floor, circumference, mean height(see above), max height 
    and possible floors
    """
    dimensions = {"area[m2] : " : round(polygon.area,1), "circumference[m] : ": round(polygon.length,1),
                  "mean height[m] : " : round(ma.mean(noZeros),1), "max height[m] : " : round(np.max(mask_chm),1),
                  "floors : " : int((np.max(mask_chm) // 3))}
    print('-> house dimensions:')
    for key,value in dimensions.items():
        print(key, value)
    return dimensions
    
def D3Plot(address, mask, dimensions):
    dims = ''
    for key,value in dimensions.items():
        dims += str(key) + str(value) +',\n'
    # plots a surface plot of the mask from polygon out of CHM
    fig = go.Figure(data=[go.Surface(z=mask)])
    fig.update_layout(title='3D house plot of ' + address + '<br><br>' + dims, width=800, height=750)
    fig.show()
    return

#### Make Plotly plot in browser for Jupyter

In [24]:
import plotly.io as pio
pio.renderers.default = "browser"

#### Check functioning

In [25]:
address = input('-> Please provide an address (or "q" to quit) : ')
while not(re.match("^([A-z-]+)\s([0-9]+[A-z]?),\s([0-9]+)\s([A-z]+)", address)): # RegEx to verify format
    print(f"Invalid address format!")
    address = input('Please provide an address (or "q" to quit) : ')
else:
    print(f"Valid address format")
    x, y = retrieve_coords(address)
    path =  retrieve_map(x,y)
    poly = retrieve_polygon(address)
    maskchm = mask_polygon(poly, path)
    dimensions = dim_house(poly , maskchm)
    D3Plot(address, maskchm, dimensions)

-> Please provide an address (or "q" to quit) : Snoekstraat 35, 9000 Gent
Valid address format
-> Lambert72 coords of provided address: 
 x=  106078.32  y=  193210.01
/home/becode/AI/3dhouse/tile_chm_k22_136.tif
[{'coordinates': [[[106087.66730889678, 193215.58208815753], [106084.20650889724, 193218.78637615964], [106073.97482889146, 193208.60884015262], [106077.30846089125, 193205.2780241482], [106086.28311689943, 193214.20519215614], [106087.66730889678, 193215.58208815753]]], 'type': 'Polygon'}]
print OK
-> mask succesful
-> house dimensions:
area[m2] :  68.4
circumference[m] :  38.5
mean height[m] :  7.8
max height[m] :  13.0
floors :  4
